### Preparing AI Ready Data
This notebook extracts statistics from 100 days of seismic data for use in classic machine learning. The environment requires obspy to be installed in addition to the standard mlgeo environment

In [1]:
#!pip install obspy
import obspy
from obspy import read
from obspy import UTCDateTime as utc
from scipy.signal import stft
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#Make folder for AI ready data
! mkdir data/ai_ready
filepath = os.getcwd() + '/data/ai_ready/'
clean_filepath = os.getcwd() + '/data/clean/'
raw_filepath = os.getcwd() + '/data/raw/'

In [3]:
#set eruption time for given sensor/volcano, example is for Sitkin Volcano
eruption_time = utc('2021-07-09T00:00:00')

In [4]:
#read in dates file from Download_Data notebook and convert to UTC objects
dates = [utc(date) for date in np.load(raw_filepath+'date_list.npy', allow_pickle=True)]
print(dates[0])

2021-06-09T00:00:00.000000Z


### Define Functions for Calculating Stats

In [5]:
# General Stats Used
def stats(data):
    mean = np.mean(data)
    maxx = np.max(data)
    minn = np.min(data)
    median = np.median(data)
    range = np.abs(maxx - minn)
    std = np.std(data)
    var = np.var(data)

    return(mean, maxx, minn, median, range, std, var)

# Spectral Domain, use STFT using the same parameters as Zahra et al. 2024
def compute_spec(data):

    x = data
    fs = 8 # specified by Zahra
    nwindow = 256 #given by Zahra
    noverlap = 32 #given by Zahra

    f, t, spectra = stft(x, fs=fs, nperseg=nwindow, noverlap=noverlap)

    #find magnitude of complex spectra
    spectra = np.abs(spectra)

    #find log 10 of spectra
    log_spectra = np.log10(spectra)



    return(spectra, log_spectra)

### Cell Below Will Save Seismograms in Day Chunks and also Combine Them into 1 Large File

In [6]:
#read in cleaned mseed files, save as daily npy files and one giant 100 day file
hundred_data = np.array([])

for i in range(60):
    i += 1

    #read mseed
    st = read(clean_filepath+str(i)+'_cleaned.mseed')

    #save data to day long npys
    data = st[0].data 
    np.save(filepath+str(i)+'_ready.npy', data)

    #append to overall array
    hundred_data = np.append(hundred_data, data)

#save hundred day data npy
np.save(filepath+'60_days.npy', hundred_data)

### Cells Below Will Compute and Save Time Series and Spectral Statistics For Each Hour of Seismogram Data
Data will also be associated with a Before or After Eruption Label in Stats file

In [7]:
#initialize statistics dataframe
statistics = ['mean','max','min','median','range','std','var',
              'spec_mean','spec_max','spec_min','spec_median','spec_range','spec_std','spec_var',
              'speclog_mean','speclog_max','speclog_min','speclog_median','speclog_range','speclog_std','speclog_var',
              'state']

stats_df = pd.DataFrame(columns=statistics)

In [8]:
for i in range(60):
    date = dates[i]
    date += 30*60 #add half an hour for determining eruption state
    i += 1

    #read mseed
    st = read(clean_filepath+str(i)+'_cleaned.mseed')
    data = st[0].data
    hz = st[0].stats.sampling_rate

    #THIS ASSUMES DATA CONTAINS 24 HOURS OF DATA PER DATA
    split_data = np.array(np.split(data, 24))

    if split_data.shape[1] == 28800:
        for hour in np.arange(24):
            date += 60*60
            if date >= eruption_time:
                state = 'after'
            elif date < eruption_time:
                state = 'before'

            data = split_data[hour]
            spectra, log_spectra = compute_spec(data)

            tmean, tmax, tmin, tmedian, trange, tstd, tvar = stats(data)

            smean, smax, smin, smedian, srange, sstd, svar = stats(spectra)

            slmean, slmax, slmin, slmedian, slrange, slstd, slvar = stats(log_spectra)

            stats_df.loc[len(stats_df)] = tmean, tmax, tmin, tmedian, trange, tstd, tvar, smean, smax, \
                smin, smedian, srange, sstd, svar, slmean, slmax, slmin, slmedian, slrange, slstd, slvar, state


In [9]:
#inspect dataframe header
stats_df.head(3)

,mean,max,min,median,range,std,var,spec_mean,spec_max,spec_min,...,spec_std,spec_var,speclog_mean,speclog_max,speclog_min,speclog_median,speclog_range,speclog_std,speclog_var,state
0,-3.491294,288.336167,-309.623540,-3.520637,597.959707,69.021092,4763.911140,3.995715,48.577898,0.005228,...,3.467311,12.022245,0.438096,1.686439,-2.281700,0.493160,3.968138,0.428269,0.183414,before
1,-4.818461,605.967566,-757.948798,-4.608615,1363.916364,71.172191,5065.480781,3.837636,76.485800,0.007620,...,3.791084,14.372317,0.409445,1.883581,-2.118037,0.460488,4.001617,0.433883,0.188255,before
2,-6.891544,547.064530,-617.394820,-6.908078,1164.459351,51.414491,2643.449882,3.107629,60.077691,0.006265,...,2.517431,6.337459,0.348699,1.778713,-2.203060,0.410225,3.981773,0.404384,0.163527,before


In [10]:
#save stats to csv file
stats_df.to_csv(path_or_buf=filepath+'statistics.csv', index=False)